In [167]:
import pandas as pd
from sklearn import preprocessing


In [217]:
prices = pd.read_pickle('nasdaq_prices.pkl')
prices = prices['Adj Close']
prices = prices.dropna(axis=1,how='any')
prices = prices.reset_index()

prices = prices.drop('Date', axis=1)

prices = prices.pct_change().iloc[1:]

prices_numpy = prices.values


min_max_scaler = preprocessing.MinMaxScaler()
prices_scaled = min_max_scaler.fit_transform(prices_numpy)


In [218]:
min_max_scaler.inverse_transform(prices_numpy)[:,1]

array([-0.28507675, -0.26016192, -0.25467409, -0.23714014, -0.24962599,
       -0.2671956 , -0.25054718, -0.24885651, -0.251042  , -0.2366159 ,
       -0.24825266, -0.28026969, -0.24932058, -0.21631821, -0.2789594 ,
       -0.28895852, -0.23217983, -0.25471199, -0.22947935, -0.27330096,
       -0.24433931, -0.21866897, -0.23661669, -0.27047928, -0.25037167,
       -0.24266509, -0.22805833, -0.23824428, -0.26051764, -0.27186306,
       -0.26519288, -0.259196  , -0.24803261, -0.2682977 , -0.30874625,
       -0.31296536, -0.27548207, -0.30308021, -0.30214877, -0.2588612 ,
       -0.28776341, -0.22698108, -0.34136736, -0.25514086, -0.30291178,
       -0.15107782, -0.28111592, -0.36686147, -0.20984003, -0.17762882,
       -0.26641068, -0.41954061, -0.32966921, -0.24644575, -0.26055268,
       -0.0147825 , -0.18220849, -0.24061113, -0.32085513, -0.33680191,
       -0.25549492, -0.33385644, -0.29133219, -0.296417  , -0.2444772 ,
       -0.20198162, -0.18021416, -0.18317358, -0.30261067, -0.22

In [221]:
prices_numpy[:,1]

array([-0.04950155, -0.0119349 , -0.00366034,  0.02277739,  0.00395119,
       -0.0225403 ,  0.00256222,  0.00511142,  0.00181614,  0.02356784,
        0.00602191, -0.04225345,  0.0044117 ,  0.05417275, -0.04027779,
       -0.05535448,  0.03025657, -0.00371749,  0.03432836, -0.03174598,
        0.01192247,  0.05062827,  0.02356666, -0.02749145,  0.00282685,
        0.01444686,  0.03647097,  0.02111257, -0.01247126, -0.02957791,
       -0.0195206 , -0.01047849,  0.0063537 , -0.02420205, -0.08519047,
       -0.09155206, -0.03503466, -0.0766472 , -0.07524277, -0.00997368,
       -0.0535525 ,  0.03809525, -0.13437667, -0.00436413, -0.07639325,
        0.15254237, -0.0435294 , -0.17281675,  0.06394057,  0.11250871,
       -0.02135679, -0.25224649, -0.11673817,  0.00874637, -0.0125241 ,
        0.35804879,  0.10560347,  0.01754383, -0.10344827, -0.12749288,
       -0.00489799, -0.12305169, -0.05893351, -0.0666004 ,  0.01171455,
        0.0757895 ,  0.10861053,  0.10414831, -0.07593923,  0.03

In [88]:
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
 
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image

In [89]:
NUM_EPOCHS = 50
LEARNING_RATE = 1e-3
BATCH_SIZE = 128
# image transformations
transform = transforms.Compose([
    transforms.ToTensor(),
])

In [16]:
trainset = datasets.FashionMNIST(
    root='./data',
    train=True, 
    download=True,
    transform=transform
)
testset = datasets.FashionMNIST(
    root='./data',
    train=False,
    download=True,
    transform=transform
)
trainloader = DataLoader(
    trainset, 
    batch_size=BATCH_SIZE,
    shuffle=True
)
testloader = DataLoader(
    testset, 
    batch_size=BATCH_SIZE, 
    shuffle=True
)

Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw


Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw
Processing...
Done!


/Users/paulrice/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /Users/distiller/project/conda/conda-bld/pytorch_1607370249289/work/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [90]:
def get_device():
    if torch.cuda.is_available():
        device = 'cuda:0'
    else:
        device = 'cpu'
    return device
def make_dir():
    image_dir = 'FashionMNIST_Images'
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
def save_decoded_image(img, epoch):
    img = img.view(img.size(0), 1, 28, 28)
    save_image(img, './FashionMNIST_Images/linear_ae_image{}.png'.format(epoch))

In [254]:
class Autoencoder(nn.Module):
    def __init__(self, feature_len):
        super(Autoencoder, self).__init__()
        # encoder
        self.enc1 = nn.Linear(in_features=feature_len, out_features=256)
        self.enc2 = nn.Linear(in_features=256, out_features=128)
        self.enc3 = nn.Linear(in_features=128, out_features=64)
        self.enc4 = nn.Linear(in_features=64, out_features=32)
        self.enc5 = nn.Linear(in_features=32, out_features=16)
        # decoder 
        self.dec1 = nn.Linear(in_features=16, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
        self.dec3 = nn.Linear(in_features=64, out_features=128)
        self.dec4 = nn.Linear(in_features=128, out_features=256)
        self.dec5 = nn.Linear(in_features=256, out_features=feature_len)
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.enc3(x))
        x = F.relu(self.enc4(x))
        x = F.relu(self.enc5(x))
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        x = F.relu(self.dec3(x))
        x = F.relu(self.dec4(x))
        x = F.relu(self.dec5(x))
        return x

In [230]:
net = Autoencoder(feature_len=784)
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

Autoencoder(
  (enc1): Linear(in_features=784, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=128, bias=True)
  (enc3): Linear(in_features=128, out_features=64, bias=True)
  (enc4): Linear(in_features=64, out_features=32, bias=True)
  (enc5): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=128, bias=True)
  (dec4): Linear(in_features=128, out_features=256, bias=True)
  (dec5): Linear(in_features=256, out_features=784, bias=True)
)


In [231]:
def train(net, trainloader, NUM_EPOCHS):
    train_loss = []
    for epoch in range(NUM_EPOCHS):
        running_loss = 0.0
        for data in trainloader:
            img, _ = data
            img = img.to(device)
            img = img.view(img.size(0), -1)
            optimizer.zero_grad()
            outputs = net(img)
            loss = criterion(outputs, img)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        loss = running_loss / len(trainloader)
        train_loss.append(loss)
        print('Epoch {} of {}, Train Loss: {:.3f}'.format(
            epoch+1, NUM_EPOCHS, loss))
        if epoch % 5 == 0:
            save_decoded_image(outputs.cpu().data, epoch)
    return train_loss
def test_image_reconstruction(net, testloader):
     for batch in testloader:
        img, _ = batch
        img = img.to(device)
        img = img.view(img.size(0), -1)
        outputs = net(img)
        outputs = outputs.view(outputs.size(0), 1, 28, 28).cpu().data
        save_image(outputs, 'fashionmnist_reconstruction.png')
        break

In [ ]:
# get the computation device
device = get_device()
print(device)
# load the neural network onto the device
net.to(device)
make_dir()
# train the network
train_loss = train(net, trainloader, NUM_EPOCHS)
plt.figure()
plt.plot(train_loss)
plt.title('Train Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.savefig('deep_ae_fashionmnist_loss.png')
# test the network
test_image_reconstruction(net, testloader)

In [27]:
type(trainloader.dataset.data)

torch.Tensor

In [31]:
trainloader.dataset.data.shape

torch.Size([60000, 28, 28])

In [165]:
total = 0
for data in trainloader:
    img, _ = data
    img = img.to(device)
    img = img.view(img.size(0), -1)
    print(img.data[0])
    break
    total+=1

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0039, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0118, 0.0000, 0.1922, 0.0627, 0.0000,
        0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0078, 0.0000, 0.0667, 0.0510, 0.0000,
        0.0078, 0.0000, 0.0000, 0.0000, 0.0000, 0.0275, 0.0000, 0.5961, 0.4314,
        0.0000, 0.0275, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0353, 0.0000, 0.5412, 0.4980,
        0.0000, 0.0235, 0.0000, 0.0000, 0.0000, 0.0000, 0.0235, 0.0000, 0.6157,
        0.6353, 0.0000, 0.0314, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
        0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0275, 0.0000, 0.6824,
        0.5294, 0.0000, 0.0314, 0.0000, 

In [35]:
total * 128

60032

In [251]:
net = Autoencoder(feature_len=prices.shape[1])
print(net)

Autoencoder(
  (enc1): Linear(in_features=2786, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=128, bias=True)
  (enc3): Linear(in_features=128, out_features=64, bias=True)
  (enc4): Linear(in_features=64, out_features=32, bias=True)
  (enc5): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=128, bias=True)
  (dec4): Linear(in_features=128, out_features=256, bias=True)
  (dec5): Linear(in_features=256, out_features=2786, bias=True)
)


In [252]:
net = Autoencoder(feature_len=prices_numpy.shape[1])
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)

Autoencoder(
  (enc1): Linear(in_features=2786, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=128, bias=True)
  (enc3): Linear(in_features=128, out_features=64, bias=True)
  (enc4): Linear(in_features=64, out_features=32, bias=True)
  (enc5): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=128, bias=True)
  (dec4): Linear(in_features=128, out_features=256, bias=True)
  (dec5): Linear(in_features=256, out_features=2786, bias=True)
)


In [276]:
net = Autoencoder(feature_len=prices_numpy.shape[1])
print(net)

criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)


torch_prices = torch.from_numpy(prices_scaled).float()
# torch_prices = torch.from_numpy(prices_numpy).float()


train_loss = []
for epoch in range(500):
    running_loss = 0.0
    
    torch_prices = torch_prices.to(device)
    optimizer.zero_grad()

    outputs = net(torch_prices)
    loss = criterion(outputs, torch_prices)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    loss = running_loss / len(trainloader)
    train_loss.append(loss)
    print('Epoch {} of {}, Train Loss: {:.10f}'.format(
        epoch+1, NUM_EPOCHS, loss))


Autoencoder(
  (enc1): Linear(in_features=2786, out_features=256, bias=True)
  (enc2): Linear(in_features=256, out_features=128, bias=True)
  (enc3): Linear(in_features=128, out_features=64, bias=True)
  (enc4): Linear(in_features=64, out_features=32, bias=True)
  (enc5): Linear(in_features=32, out_features=16, bias=True)
  (dec1): Linear(in_features=16, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
  (dec3): Linear(in_features=64, out_features=128, bias=True)
  (dec4): Linear(in_features=128, out_features=256, bias=True)
  (dec5): Linear(in_features=256, out_features=2786, bias=True)
)
Epoch 1 of 50, Train Loss: 0.0004598195
Epoch 2 of 50, Train Loss: 0.0004510943
Epoch 3 of 50, Train Loss: 0.0004409455
Epoch 4 of 50, Train Loss: 0.0004290288
Epoch 5 of 50, Train Loss: 0.0004150474
Epoch 6 of 50, Train Loss: 0.0003983775
Epoch 7 of 50, Train Loss: 0.0003789245
Epoch 8 of 50, Train Loss: 0.0003560928
Epoch 9 of 50, Train Loss: 0.0003296116
Epo

Epoch 189 of 50, Train Loss: 0.0001547688
Epoch 190 of 50, Train Loss: 0.0001541002
Epoch 191 of 50, Train Loss: 0.0001534637
Epoch 192 of 50, Train Loss: 0.0001527567
Epoch 193 of 50, Train Loss: 0.0001518481
Epoch 194 of 50, Train Loss: 0.0001509607
Epoch 195 of 50, Train Loss: 0.0001503781
Epoch 196 of 50, Train Loss: 0.0001497978
Epoch 197 of 50, Train Loss: 0.0001490250
Epoch 198 of 50, Train Loss: 0.0001485606
Epoch 199 of 50, Train Loss: 0.0001479874
Epoch 200 of 50, Train Loss: 0.0001475447
Epoch 201 of 50, Train Loss: 0.0001472019
Epoch 202 of 50, Train Loss: 0.0001468336
Epoch 203 of 50, Train Loss: 0.0001466629
Epoch 204 of 50, Train Loss: 0.0001463931
Epoch 205 of 50, Train Loss: 0.0001462633
Epoch 206 of 50, Train Loss: 0.0001460790
Epoch 207 of 50, Train Loss: 0.0001459054
Epoch 208 of 50, Train Loss: 0.0001457667
Epoch 209 of 50, Train Loss: 0.0001455429
Epoch 210 of 50, Train Loss: 0.0001453102
Epoch 211 of 50, Train Loss: 0.0001451368
Epoch 212 of 50, Train Loss: 0.000

Epoch 385 of 50, Train Loss: 0.0001110329
Epoch 386 of 50, Train Loss: 0.0001109571
Epoch 387 of 50, Train Loss: 0.0001109113
Epoch 388 of 50, Train Loss: 0.0001108897
Epoch 389 of 50, Train Loss: 0.0001108875
Epoch 390 of 50, Train Loss: 0.0001108984
Epoch 391 of 50, Train Loss: 0.0001109094
Epoch 392 of 50, Train Loss: 0.0001109095
Epoch 393 of 50, Train Loss: 0.0001108845
Epoch 394 of 50, Train Loss: 0.0001108478
Epoch 395 of 50, Train Loss: 0.0001107982
Epoch 396 of 50, Train Loss: 0.0001107283
Epoch 397 of 50, Train Loss: 0.0001106356
Epoch 398 of 50, Train Loss: 0.0001105383
Epoch 399 of 50, Train Loss: 0.0001104464
Epoch 400 of 50, Train Loss: 0.0001103618
Epoch 401 of 50, Train Loss: 0.0001102863
Epoch 402 of 50, Train Loss: 0.0001102255
Epoch 403 of 50, Train Loss: 0.0001101823
Epoch 404 of 50, Train Loss: 0.0001101144
Epoch 405 of 50, Train Loss: 0.0001099731
Epoch 406 of 50, Train Loss: 0.0001097890
Epoch 407 of 50, Train Loss: 0.0001096001
Epoch 408 of 50, Train Loss: 0.000

KeyboardInterrupt: 

In [277]:
outputs = net(torch_prices)

outputs = outputs.cpu().data


In [266]:
inversed = min_max_scaler.inverse_transform(outputs)

In [267]:
outputs.shape

torch.Size([234, 2786])

In [278]:
outputs[:,1]

tensor([0.3534, 0.3747, 0.3665, 0.3797, 0.3779, 0.3623, 0.3892, 0.3782, 0.3782,
        0.4070, 0.3645, 0.3507, 0.3673, 0.3668, 0.3355, 0.3303, 0.3934, 0.3486,
        0.3631, 0.3139, 0.3936, 0.4059, 0.4070, 0.3661, 0.3387, 0.3824, 0.3918,
        0.3889, 0.3713, 0.3586, 0.3645, 0.3887, 0.3768, 0.3422, 0.2887, 0.2817,
        0.3447, 0.2730, 0.3376, 0.4431, 0.3096, 0.4447, 0.2773, 0.3089, 0.1565,
        0.4456, 0.2224, 0.2011, 0.5372, 0.2978, 0.5298, 0.2737, 0.5558, 0.3000,
        0.3138, 0.5934, 0.4291, 0.5350, 0.2579, 0.4301, 0.3710, 0.2072, 0.3994,
        0.2906, 0.5540, 0.3857, 0.4782, 0.4796, 0.3140, 0.4201, 0.2706, 0.3480,
        0.4834, 0.3320, 0.3075, 0.4098, 0.3978, 0.4068, 0.4793, 0.4029, 0.5020,
        0.2871, 0.2755, 0.3771, 0.3669, 0.3454, 0.4146, 0.4614, 0.3437, 0.2843,
        0.2807, 0.3765, 0.4061, 0.5205, 0.3171, 0.4587, 0.3612, 0.3860, 0.4552,
        0.4577, 0.3143, 0.3621, 0.3817, 0.3937, 0.4504, 0.3881, 0.4705, 0.4339,
        0.3294, 0.3027, 0.1833, 0.4323, 

In [279]:
prices_scaled[:,0]

array([0.58477941, 0.34796427, 0.40346791, 0.5175801 , 0.32241814,
       0.44566968, 0.45859635, 0.44372618, 0.48242622, 0.42880644,
       0.11564978, 0.41803757, 0.57706403, 0.28348247, 0.29014816,
       0.32847683, 0.32561476, 0.2415826 , 0.40346791, 0.40346791,
       0.40346791, 0.38588383, 0.42120488, 0.36829976, 0.20664947,
       0.5420921 , 0.3663815 , 0.34680789, 0.50059921, 0.49618392,
       0.65178581, 0.2137233 , 0.52550459, 0.1896227 , 0.2932108 ,
       0.57830407, 0.38557534, 0.27711134, 0.34573906, 0.30445079,
       0.40346791, 0.44509551, 0.32187769, 0.48845772, 0.34227533,
       0.42449628, 0.36184018, 0.27598325, 0.31281224, 0.37974985,
       0.54745057, 0.        , 0.43140972, 0.84449578, 0.26748427,
       0.45203359, 0.5220583 , 0.60520179, 0.32187769, 0.38222048,
       0.40346791, 0.40346791, 0.33905467, 0.46998157, 0.40346791,
       0.29611238, 0.35814007, 0.33393089, 0.3314765 , 0.40346791,
       0.50296817, 0.37974985, 0.37947068, 0.37918514, 0.25601